In [ ]:
# General imports
import os
import sys
import pandas as pd

pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import subprocess

# Other imports
import multiprocessing
import parmap
import collections
from tqdm import tqdm

tqdm.pandas()

import json
from pandarallel import pandarallel

pandarallel.initialize(nb_workers=60, progress_bar=True)
from pprint import pprint

# Custom utils
sys.path.append("/home/weber/PycharmProjects/EXOTIC/src")
from utils.utils import load_config_file

# Figures imports
import matplotlib

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from statannot import add_stat_annotation
import matplotlib.font_manager as font_manager

# Font settings
font_dirs = ['/home/weber/Fonts', ]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
font_list = font_manager.createFontList(font_files)
font_manager.fontManager.ttflist.extend(font_list)

from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']
rcParams['font.weight'] = 'light'

## YAML FILES CONFIG
files = load_config_file(config_file="/home/weber/PycharmProjects/EXOTIC/src/config_clean.yaml")

dicts = json.load(open("/home/weber/PycharmProjects/EXOTIC/src/EXOTIC_config.json"))


# Phenotypes

## OMIM raw file

In [ ]:
omim_raw = pd.read_csv('/gstock/EXOTIC/data/PHENOTYPES/mim2gene.txt', sep='\t', skiprows=4)
omim_raw.columns = ['MIM', 'Type', 'Gene_ID', 'HGNC_symbol', 'ENSG']
omim_raw = omim_raw.loc[omim_raw['Type'] == 'gene']
omim_raw = omim_raw.drop_duplicates().sort_values(by='HGNC_symbol')
omim_raw = omim_raw.dropna(subset=['HGNC_symbol'])
omim_raw

In [ ]:
omim_pheno = pd.read_csv('/gstock/EXOTIC/data/PHENOTYPES/genemap2.txt', sep='\t', skiprows=3)

omim_pheno.columns = ['CHR', 'Start', 'End', 'Cyto', 'Computed_cyto', 'MIM', 'Gene_symbol', 'Gene_descr', 'Approved_symbol', 'Gene_ID', 'ENSG', 'Comments', 'Phenotype', 'Mouse_gene_ID']
omim_pheno = omim_pheno.loc[(omim_pheno['MIM'].isna() == False) & (omim_pheno['Phenotype'].isna() == False)]
omim_pheno['Phenotype'] = omim_pheno['Phenotype'].astype(str)

omim_pheno = omim_pheno.loc[(omim_pheno['ENSG'].isna() == False)]
omim_pheno['MIM'] = omim_pheno['MIM'].astype(int)
# omim_pheno['Gene_symbol'] = omim_pheno['Gene_symbol'].apply(lambda r: r.split(',')[0])
# omim_pheno = omim_pheno.loc[omim_pheno['Phenotype'].str.contains('\[')]
# omim_raw = omim_raw.loc[omim_raw['Type'] == 'gene']
# omim_raw = omim_raw.drop_duplicates().sort_values(by='HGNC_symbol')
# omim_raw = omim_raw.dropna(subset=['HGNC_symbol'])
omim_pheno

In [ ]:
omim_pheno.ENSG.nunique()

## Processed phenotypes through API

In [ ]:
from Phenotypes import omim_associations_exotic
omim = omim_associations_exotic.load_omim(files["EXOTIC"]["omim_detailed"], files["BIOMART"]["biomart_omim"])
# omim.head()
omim

In [ ]:
omim

In [ ]:
omim.OMIM.nunique()

## OMIM entries with multiple phenotypes & at least one rare

In [ ]:
def load_omim(omim_path, biomart_omim_path):
    omim = pd.read_csv(
        omim_path,
        compression="gzip",
        sep="\t",
    )
#     return omim

    print("Total : ", omim.OMIM.nunique())
    omim = omim.dropna(subset=list(omim.columns[6:-2]), how="all")
    print("Dropna on all cols : ", omim.OMIM.nunique())

    biomart_omim = pd.read_csv(biomart_omim_path, sep="\t", compression="gzip").dropna(subset=["MIM gene accession"])
    biomart_omim["MIM gene accession"] = biomart_omim["MIM gene accession"].astype(int)
    biomart_omim = biomart_omim.rename({"MIM gene accession": "OMIM", "Gene stable ID": "ensg", "Gene name": "Name"}, axis=1)

    ## ADD Gene TO OMIM
    omim = pd.merge(biomart_omim[["ensg", "Name", "OMIM"]], omim, on="OMIM")
    print("Merge BIOMART : ", omim.OMIM.nunique())


    # MELT
    omim = omim.melt(id_vars=list(omim.columns)[:7], value_vars=list(omim.columns)[7:], var_name="OMIM_BP", value_name="OMIM_BP_phenotypes").dropna()

    return omim
omim = load_omim(files["EXOTIC"]["omim_detailed"], files["BIOMART"]["biomart_omim"])
omim

In [ ]:
l = list()
for gene in tqdm(omim.Name.unique()):

    gene_omim = omim.loc[omim['Name'] == gene]
    if gene_omim.Pheno_OMIM.nunique() > 1:
#         print(gene, gene_omim.shape[0])
        gene_omim_bp = gene_omim[['Name', 'Pheno_OMIM', 'OMIM_BP']].groupby(['Name', 'Pheno_OMIM'])['OMIM_BP'].apply(set).reset_index()
        counter_pheno = collections.Counter([sub_e for e in gene_omim_bp['OMIM_BP'].values.tolist() for sub_e in e])
        counter_pheno_rare = [k for k,v in counter_pheno.items() if v == 1]
        l.append(gene_omim.loc[gene_omim['OMIM_BP'].isin(counter_pheno_rare)])
        
#         for pheno_rare in counter_pheno_rare:
#             print(gene_omim.loc[gene_omim['OMIM_BP'] == pheno_rare, ['Pheno_OMIM', 'Pheno_prefered_title', 'OMIM_BP', 'OMIM_BP_phenotypes']].values.tolist())
omim_multi_pheno_filterred = pd.concat(l)
omim_multi_pheno_filterred

In [ ]:
omim_multi_pheno_filterred.loc[(omim_multi_pheno_filterred['Name'] == 'NF1') & (omim_multi_pheno_filterred['OMIM_BP'] == 'genitourinaryInternalGenitaliaMale'), 'OMIM_BP_phenotypes'].values.tolist()[0]

In [ ]:
omim_multi_pheno_filterred.loc[(omim_multi_pheno_filterred['Name'] == 'NF1'), 'OMIM_BP_phenotypes'].values.tolist()[4]

In [ ]:
omim_multi_pheno_filterred.Name.nunique()

In [ ]:
for col in ['ensg', 'Name', 'OMIM']:
    print(col, omim[col].nunique())

## EXOTIC imports

In [ ]:
exotic = pd.read_parquet(files["EXOTIC"]["exotic_modified_zscore"])
exotic[['Exon_start', 'Exon_stop']] = exotic['Exon'].str.split('-', expand=True)
# exotic_up = omim_associations_exotic.load_exotic(files["EXOTIC"]["exotic_modified_zscore"], 'up', cutoff=0.5)
# exotic_down = omim_associations_exotic.load_exotic(files["EXOTIC"]["exotic_modified_zscore"], 'down', cutoff=0.5)

In [ ]:
from matplotlib_venn import venn2
import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))
plt.rcParams.update({"font.size" : 17})
venn2([set(exotic.symbol.unique().tolist()), set(omim_pheno.Approved_symbol.unique().tolist())], set_labels=['EXOTIC genes', 'OMIM genes with phenotypes'], set_colors=['purple', 'skyblue'], alpha=0.5)


In [ ]:
omim

In [ ]:
def compare_to_omim_basic_exon_level(exotic, min_max, omim, omim_detailed=True):
    print(min_max)
    print(exotic.symbol.nunique())
    print(exotic.MAP.nunique())

    mapping_omim_gtex = dicts["mapping_omim_gtex_detailed"]

    omim = omim.where(pd.notnull(omim), None)

    omim = omim.loc[~omim.duplicated(keep="last", subset=["OMIM", "Pheno_OMIM", "Pheno_prefered_title"])]

#     print(omim)

#     print(exotic)

    merge = pd.merge(omim, exotic, on="ensg")
    merge["EXOTIC_tissue_BP"] = merge["EXOTIC_tissues_above_cutoff_{}".format(min_max)].map(mapping_omim_gtex)
    merge = merge.explode("EXOTIC_tissue_BP")
    merge = merge[
        [
            "ensg",
            "symbol",
            "MAP",
            "OMIM",
            "OMIM_BP",
            "OMIM_BP_phenotypes",
            "EXOTIC_tissues_above_cutoff_{}".format(min_max),
            "EXOTIC_{}".format(min_max),
            "EXOTIC_bins_{}".format(min_max),
            "EXOTIC_tissue_BP",
        ]
    ]

#     print(merge)
    print(merge.loc[merge["OMIM_BP"] == merge["EXOTIC_tissue_BP"], 'symbol'].nunique())
    print(merge.loc[merge["OMIM_BP"] == merge["EXOTIC_tissue_BP"], 'MAP'].nunique())

In [ ]:
compare_to_omim_basic_exon_level(exotic_up, 'up', omim)

In [ ]:
compare_to_omim_basic_exon_level(exotic_down, 'down', omim)

# Clinvar

In [ ]:
clinvar = pd.read_parquet(files['EXOTIC']['clinvar_file_path'])
clinvar = clinvar.loc[(clinvar['Status'] == 'Pathogenic') & (clinvar['Real_Status'].str.contains('onflict') == False)]
# clinvar = clinvar.loc[clinvar['OMIM_VARIANT_ID'] != ""]
# clinvar['ALT_Lite'] = clinvar['ALT'].apply(lambda r: eval(r)[0]) 
# clinvar['VAR_ID_Lite'] = clinvar['CHROM'].astype(str) + '_' + clinvar['POS'].astype(str) + '_' + clinvar['REF'].astype(str) + '_' + clinvar['ALT_Lite'].astype(str)

clinvar

In [ ]:
from matplotlib_venn import venn2
import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))
plt.rcParams.update({"font.size" : 17})
venn2([
    set(exotic.symbol.unique().tolist()), 
    set(clinvar.GENE.unique().tolist())
], 
    set_labels=['EXOTIC genes', 'ClinVar Pathogenic (RV)'], 
    set_colors=['purple', 'green'], alpha=0.5)


In [ ]:
from matplotlib_venn import venn2
import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))
plt.rcParams.update({"font.size" : 17})
venn2([
    set(omim_pheno.Approved_symbol.unique().tolist()),
    set(clinvar.GENE.unique().tolist())
], 
    set_labels=['OMIM genes with phenotypes', 'ClinVar Pathogenic (RV)'], 
    set_colors=['skyblue', 'green'], alpha=0.5)


In [ ]:
from matplotlib_venn import venn3
import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))
plt.rcParams.update({"font.size" : 17})
venn3([
    set(exotic.symbol.unique().tolist()), 
    set(omim_pheno.Approved_symbol.unique().tolist()),
    set(clinvar.GENE.unique().tolist())
], 
    set_labels=['EXOTIC genes', 'OMIM genes with phenotypes', 'ClinVar (P/LP with OMIM IDs)'], 
    set_colors=['purple', 'skyblue', 'green'], alpha=0.5)


In [ ]:
exotic.head()

In [ ]:
clinvar.tail()

In [ ]:
def mp_variants(gene, exotic, clinvar, l):
    exotic_tmp_gene = exotic.loc[exotic['symbol'] == gene]
    clinvar_tmp_gene = clinvar.loc[clinvar['GENE'] == gene]
    for exon in exotic_tmp_gene.Exon.unique().tolist():
        match_variants = clinvar_tmp_gene.POS.between(int(exon.split('-')[0]), int(exon.split('-')[1]))
        match_variants = clinvar_tmp_gene.loc[match_variants.loc[match_variants == True].index.tolist(), 'alleleid'].values.tolist()
        if match_variants:
            l.append({exon : match_variants})
        
# for gene in tqdm():
m = multiprocessing.Manager()
l = m.list()
genes = exotic.symbol.unique().tolist()
parmap.starmap(mp_variants, list(zip(genes)), exotic, clinvar, l, pm_pbar=True)
map_d = {k: v for d in l for k, v in d.items()}
exotic['AlleleIDs_clinvar'] = exotic['Exon'].map(map_d)
# exotic_up['AlleleIDs_clinvar'] = exotic_up['Exon'].map(map_d)
# exotic_down['AlleleIDs_clinvar'] = exotic_down['Exon'].map(map_d)


# print(l)

In [ ]:
exotic.loc[exotic['AlleleIDs_clinvar'].isna() == False, 'symbol'].nunique()

In [ ]:
exotic_clinvar = pd.merge(exotic.explode('AlleleIDs_clinvar').rename({'AlleleIDs_clinvar' : 'alleleid'}, axis=1), clinvar, on='alleleid')
exotic_clinvar.to_csv('/gstock/EXOTIC/data/VARIATIONS/exotic_clinvar.csv.gz', compression='gzip', sep='\t')
exotic_clinvar

In [ ]:
exotic['AlleleIDs_clinvar'] = exotic['Exon'].map(map_d)
exotic_up['AlleleIDs_clinvar'] = exotic_up['Exon'].map(map_d)
exotic_down['AlleleIDs_clinvar'] = exotic_down['Exon'].map(map_d)


In [ ]:
exotic_clinvar = exotic.loc[exotic['AlleleIDs_clinvar'].isna() == False]
print(exotic_clinvar.symbol.nunique())
print(exotic_clinvar.MAP.nunique())

In [ ]:
from matplotlib_venn import venn3
import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))
plt.rcParams.update({"font.size" : 17})
venn3([
    set(exotic_clinvar.symbol.unique().tolist()), 
    set(omim_pheno.Approved_symbol.unique().tolist()),
    set(clinvar.GENE.unique().tolist())
], 
    set_labels=['EXOTIC genes with\npathogenic variants\ninside EXOTIC exon', 'OMIM genes with phenotypes', 'ClinVar Pathogenic (RV)'], 
    set_colors=['purple', 'skyblue', 'green'], alpha=0.5)


In [ ]:
exotic_up[['symbol', 'EXOTIC_bins_{}'.format(min_max)]].drop_duplicates()['EXOTIC_bins_{}'.format(min_max)].value_counts().sort_index()

In [ ]:
exotic_up[['MAP', 'EXOTIC_bins_{}'.format(min_max)]].drop_duplicates()['EXOTIC_bins_{}'.format(min_max)].value_counts().sort_index()

In [ ]:
min_max = 'down'
concat_up_stats_exotic_clinvar = pd.concat([
    exotic_up[['symbol', 'EXOTIC_bins_{}'.format(min_max)]].drop_duplicates()['EXOTIC_bins_{}'.format(min_max)].value_counts().sort_index(),
    exotic_up[['MAP', 'EXOTIC_bins_{}'.format(min_max)]].drop_duplicates()['EXOTIC_bins_{}'.format(min_max)].value_counts().sort_index(),
    exotic_up.loc[exotic_up['AlleleIDs_clinvar'].isna() == False][['symbol', 'EXOTIC_bins_{}'.format(min_max)]].drop_duplicates()['EXOTIC_bins_{}'.format(min_max)].value_counts().sort_index(),
    exotic_up.loc[exotic_up['AlleleIDs_clinvar'].isna() == False][['MAP', 'EXOTIC_bins_{}'.format(min_max)]].drop_duplicates()['EXOTIC_bins_{}'.format(min_max)].value_counts().sort_index(),
], axis=1
)
concat_up_stats_exotic_clinvar.columns = ['EXOTIC genes', 'EXOTIC exons', 'Clinvar X EXOTIC genes', 'Clinvar X EXOTIC exons']
cols = concat_up_stats_exotic_clinvar.columns
concat_up_stats_exotic_clinvar

In [ ]:
def show_values_on_bars(axs, i=0, fontsize=10, rotation=0, color='black', pad=0):
    def _show_on_single_plot(ax):
        for p in ax.patches:
            print(p)
            _x = p.get_x() + p.get_width()/2 
            _y = p.get_y() + (p.get_height()) + pad
            if i == 0:
                value = "{:,.0f}".format(p.get_height())
            if i == 2:
                value = "{:.2f}".format(p.get_height())

            if i == 3:
                value = "{:.3f}".format(p.get_height())
            if p.get_height() > 0:
                ax.text(_x, _y, value, ha="center", fontsize=fontsize, rotation=rotation, color=color)

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax)
    else:
        _show_on_single_plot(axs)

plt.rcParams.update({"font.size" : 20, 'text.color' : 'black', 'axes.labelcolor' : 'black'})
min_max = 'down'
sns.set_context('paper', font_scale=2)
f, ax = plt.subplots(figsize=(15,15))
# data = concat_up_stats_exotic_clinvar.reset_index()
data = concat_down_stats_exotic_clinvar.reset_index()

concat_down_stats_exotic_clinvar
data.columns = ['EXOTIC_bins'] + list(cols)
data = data.melt(id_vars=['EXOTIC_bins'], value_vars=cols)


sns.barplot(data=data, x='EXOTIC_bins', y='value', hue='variable', palette='Reds', ax=ax)
plt.yscale('log')
plt.legend(title='')
ax.set_ylabel('Count', labelpad=0.2)
ax.set_xlabel('EXOTIC-{}'.format(min_max), labelpad=3)
ax.spines['top'].set_linewidth(0)
ax.spines['right'].set_linewidth(0)
ax.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax.ticklabel_format(axis='y', useOffset=False)
ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
# plt.xlabel('TEST')
ax.set_ylim(ymin=2, ymax=1e4)
ax.grid(axis='y', alpha=0.4)
show_values_on_bars(ax)
plt.tight_layout()


In [ ]:
concat_up_stats_exotic_clinvar.reset_index()

In [ ]:
concat_up_stats_exotic_clinvar_tmp = concat_up_stats_exotic_clinvar.reset_index()
concat_up_stats_exotic_clinvar_tmp.columns = ['EXOTIC_bins'] + list(cols)
concat_up_stats_exotic_clinvar_tmp['EXOTIC_bin_start'] = concat_up_stats_exotic_clinvar_tmp['EXOTIC_bins'].apply(lambda r: r.split(' - ')[0])
concat_up_stats_exotic_clinvar_tmp = concat_up_stats_exotic_clinvar_tmp.melt(id_vars=['EXOTIC_bin_start'], value_vars=cols)


sns.set_context('paper', font_scale=2)
f, ax = plt.subplots(figsize=(10,10))
data = concat_up_stats_exotic_clinvar_tmp.reset_index()
ax1_ = ax.bar(data.EXOTIC_bin_start.values, data.snpId_total.values,
#        yerr=error,
       align='edge',
       
       color='#8491B4FF',
       width=-0.020
      )
ax2 = ax.twinx()
ax2_ = ax2.bar(data.EXOTIC_bin_start.values, data.snpId.values, color='#91D1C2FF', align='edge', width=0.02)
ax.yaxis.label.set_color('#8491B4FF')
ax2.yaxis.label.set_color('#91D1C2FF')

ax.set_xticks(np.arange(min(data.EXOTIC_bin_start), max(data.EXOTIC_bin_start)+0.05, 0.05))


tkw = dict(size=4, width=1.5)
ax.tick_params(axis='y', colors='#8491B4FF', **tkw)
ax2.tick_params(axis='y', colors='#91D1C2FF', **tkw)
ax.set_ylabel('All sQTLs')
ax.set_xlabel('EXOTIC-max score')
ax2.set_ylabel('Tissue-specific sQTLs')
ax.spines['top'].set_linewidth(0)
ax2.spines['top'].set_linewidth(0)
# plt.title('Up')

In [ ]:
min_max = 'down'
concat_down_stats_exotic_clinvar = pd.concat([
    exotic_down[['symbol', 'EXOTIC_bins_{}'.format(min_max)]].drop_duplicates()['EXOTIC_bins_{}'.format(min_max)].value_counts().sort_index(),
    exotic_down[['MAP', 'EXOTIC_bins_{}'.format(min_max)]].drop_duplicates()['EXOTIC_bins_{}'.format(min_max)].value_counts().sort_index(),
    exotic_down.loc[exotic_down['AlleleIDs_clinvar'].isna() == False][['symbol', 'EXOTIC_bins_{}'.format(min_max)]].drop_duplicates()['EXOTIC_bins_{}'.format(min_max)].value_counts().sort_index(),
    exotic_down.loc[exotic_down['AlleleIDs_clinvar'].isna() == False][['MAP', 'EXOTIC_bins_{}'.format(min_max)]].drop_duplicates()['EXOTIC_bins_{}'.format(min_max)].value_counts().sort_index(),
], axis=1
)
concat_down_stats_exotic_clinvar.columns = ['EXOTIC genes', 'EXOTIC exons', 'Clinvar X EXOTIC genes', 'Clinvar X EXOTIC exons']
concat_down_stats_exotic_clinvar

In [ ]:
f, ax = plt.subplots(figsize=(15,15))
data = concat_down_stats_exotic_clinvar.reset_index()
data.columns = ['EXOTIC_bins'] + list(cols)
data = data.melt(id_vars=['EXOTIC_bins'], value_vars=cols)
palette = ['#DC0000', '#E64B35',"#fab1a0", "#fd79a8"]
min_max = 'down'

sns.barplot(data=data, x='EXOTIC_bins', y='value', hue='variable', palette='Reds')
plt.legend(title='')
plt.yscale('log')
ax.spines['top'].set_linewidth(0)
ax.spines['right'].set_linewidth(0)
import matplotlib
ax.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax.ticklabel_format(axis='y', useOffset=False)
ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.set_ylabel('Count')
ax.set_xlabel('EXOTIC-{}'.format(min_max))
ax.set_ylim(ymin=2, ymax=1e4)
# ax.set_yticks()

## ClinVar PMID

In [ ]:
clinvar_pmid = pd.read_csv(files['EXOTIC']['clinvar_pmid_mapping_path'], sep='\t')
clinvar_pmid

In [ ]:
clinvar_pmid.loc[clinvar_pmid['#AlleleID'].isin(exotic_clinvar.explode('AlleleIDs_clinvar').AlleleIDs_clinvar.unique().tolist()), 'citation_id'].unique().tolist()

In [ ]:
exotic.symbol.nunique()

In [ ]:
exotic_clinvar.explode('AlleleIDs_clinvar')

In [ ]:
def return_pmid(r):
    l = list()
    for e in r:
        pmids = clinvar_pmid.loc[clinvar_pmid['#AlleleID'] == e, 'citation_id'].values.tolist()
        if pmids :
            l.append(pmids[0])
    return l 

exotic_clinvar['citation_id'] = exotic_clinvar.explode('AlleleIDs_clinvar').AlleleIDs_clinvar.apply(lambda r: return_pmid(r))
exotic_clinvar

In [ ]:
exotic.loc[exotic['symbol'] == 'SPRED1']

In [ ]:
exotic_clinvar_exploded_variants = exotic_clinvar.explode('AlleleIDs_clinvar')
exotic_clinvar_exploded_variants = pd.merge(exotic_clinvar_exploded_variants.drop('citation_id', axis=1), clinvar_pmid[['#AlleleID', 'citation_id']], left_on='AlleleIDs_clinvar', right_on='#AlleleID').drop('#AlleleID', axis=1)

exotic_clinvar_exploded_variants

In [ ]:
exotic_clinvar_exploded_pmids = exotic_clinvar.explode('citation_id').dropna(subset=['citation_id'])
exotic_clinvar_exploded_pmids

In [ ]:
exotic_clinvar_exploded_pmids.loc[exotic_clinvar_exploded_pmids['symbol'] == 'SPRED1', 'citation_id'].unique()

In [ ]:
exotic_clinvar_exploded_pmids.citation_id.nunique()

In [ ]:
import requests
pmid = "25264593"
pubtator_cat = "pm"
url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?{}ids={}".format(pubtator_cat, pmid)
r = requests.get(url)
j = r.json()
title = j['passages'][0]['text']
title

In [ ]:
import requests

def retrieve_pmid_infos(pmid, l):
#     pmid = "31009165"
    pubtator_cat = "pm"
    url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?{}ids={}".format(pubtator_cat, pmid)
    try:
        r = requests.get(url)
        j = r.json()
        title = j['passages'][0]['text']
        abstract = j['passages'][1]['text']
        l.append({
            'citation_id' : pmid,
            'title' : title,
            'abstract' : abstract,
            }
        )
    except:
        print(pmid)

pmids = clinvar_pmid.loc[clinvar_pmid['#AlleleID'].isin(exotic_clinvar.explode('AlleleIDs_clinvar').AlleleIDs_clinvar.unique().tolist()), 'citation_id'].unique().tolist()
pmids = [e for e in pmids if 'NBK' not in e]
m = multiprocessing.Manager()
l = m.list()

parmap.starmap(retrieve_pmid_infos, list(zip(pmids)), l, pm_pbar=True)
    
pmids_infos = pd.DataFrame(list(l))
pmids_infos

In [ ]:
exotic_clinvar_pmid_lite

In [ ]:
import requests

def retrieve_pmid_infos(pmid, l):
#     pmid = "31009165"
    pubtator_cat = "pm"
    url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?{}ids={}".format(pubtator_cat, pmid)
    try:
        r = requests.get(url)
        j = r.json()
        title = j['passages'][0]['text']
        abstract = j['passages'][1]['text']
        l.append({
            'citation_id' : pmid,
            'title' : title,
            'abstract' : abstract,
            }
        )
    except:
        print(pmid)

pmids = exotic_clinvar_pmid.loc[exotic_clinvar_pmid['title'].isna() == True, 'citation_id'].unique().tolist()
pmids = [e for e in pmids if 'NBK' not in e]
m = multiprocessing.Manager()
l = m.list()

parmap.starmap(retrieve_pmid_infos, list(zip(pmids)), l, pm_pbar=True)
    
pmids_infos_second = pd.DataFrame(list(l))
pmids_infos_second

In [ ]:
pd.concat([pmids_infos, pmids_infos_second])

In [ ]:
exotic_clinvar_pmid.columns

In [ ]:
exotic_clinvar_pmid = pd.merge(exotic_clinvar_exploded_variants, pd.concat([pmids_infos, pmids_infos_second]), how='left', on='citation_id')
exotic_clinvar_pmid = exotic_clinvar_pmid.loc[exotic_clinvar_pmid['citation_id'].isin(citation_to_keep)]
exotic_clinvar_pmid['AlleleIDs_clinvar'] = exotic_clinvar_pmid['AlleleIDs_clinvar'].astype(str)

exotic_clinvar_pmid_lite = exotic_clinvar_pmid[['MAP', 'citation_id', 'AlleleIDs_clinvar']].groupby(['MAP', 'citation_id'])['AlleleIDs_clinvar'].apply(lambda r: ','.join(r))
exotic_clinvar_pmid_lite = exotic_clinvar_pmid_lite.reset_index().drop_duplicates()


exotic_clinvar_pmid = pd.merge(exotic_clinvar_pmid.drop(['AlleleIDs_clinvar'], axis=1).drop_duplicates(subset=['MAP', 'citation_id']), exotic_clinvar_pmid_lite, on=['MAP', 'citation_id'])

# COMPUTE BINS
r = np.arange(0.5,1.05,0.1)
bins = r
labels = bins.copy()
labels_ratio = [str(round(labels[j], 3)) + " - " + str(round(labels[j + 1], 3)) for j in range(len(labels) - 1)]
for min_max in ['up', 'down']:
    for j, b in enumerate(labels_ratio):
        if j > 0:
            b_start = float(b.split(' - ')[0])
            b_end = float(b.split(' - ')[1])
            print(b_start)
#             print(len(exotic_clinvar_pmid[list(exotic_clinvar_pmid.columns)[9:9+53]].columns))
#             print(exotic_clinvar_pmid[list(exotic_clinvar_pmid.columns)[9:9+53]].columns)
            if min_max == 'up':
                exotic_clinvar_pmid['EXOTIC_{}_{}'.format(b, min_max)] = exotic_clinvar_pmid[list(exotic_clinvar_pmid.columns)[9:9+53]].apply(lambda r: [exotic_clinvar_pmid.columns[9:9+53][i] for i, e in enumerate(r) if float(e) >= b_start and float(e) < b_end], axis=1)
            elif min_max == 'down':
                exotic_clinvar_pmid['EXOTIC_{}_{}'.format(b, min_max)] = exotic_clinvar_pmid[list(exotic_clinvar_pmid.columns)[9:9+53]].apply(lambda r: [exotic_clinvar_pmid.columns[9:9+53][i] for i, e in enumerate(r) if (1-float(e)) >= b_start and (1-float(e)) < b_end], axis=1)

                
exotic_clinvar_pmid['abstract'] = exotic_clinvar_pmid['abstract'].astype(str)

exotic_clinvar_pmid['Isoform_abstract'] = exotic_clinvar_pmid['abstract'].apply(lambda r: "isoform" in r.lower())
# exotic_clinvar_pmid = exotic_clinvar_pmid.loc[exotic_clinvar_pmid['Isoform_abstract'] == True]
exotic_clinvar_pmid = exotic_clinvar_pmid.loc[exotic_clinvar_pmid['symbol'] == 'NF1']


exotic_clinvar_pmid = exotic_clinvar_pmid[[
    'symbol', 'Exon', 'EXOTIC_0.6 - 0.7_up', 'EXOTIC_0.7 - 0.8_up', 'EXOTIC_0.8 - 0.9_up',
       'EXOTIC_0.9 - 1.0_up', 'EXOTIC_0.6 - 0.7_down', 'EXOTIC_0.7 - 0.8_down',
       'EXOTIC_0.8 - 0.9_down', 'EXOTIC_0.9 - 1.0_down', 'AlleleIDs_clinvar', 'citation_id', 'title', 'abstract', 'Isoform_abstract'
]]
exotic_clinvar_pmid.to_excel('/gstock/EXOTIC/data/PMIDS/NF1_EXOTIC_clinvar_PMIDS.xlsx', index=False)
exotic_clinvar_pmid


# exotic_clinvar_pmid[['symbol', 'citation_id']].drop_duplicates().symbol.value_counts()

# exotic_clinvar_pmid[['symbol', 'ensg', 'Exon', 'EXOTIC_bins_up', 'EXOTIC_tissues_up', 'EXOTIC_bins_down', 'EXOTIC_tissues_down', 'AlleleIDs_clinvar', 'citation_id', "title", 'abstract']].to_excel('/gstock/EXOTIC/data/PMIDS/EXOTIC_clinvar_PMIDS.xlsx', index=False)


In [ ]:
exotic_clinvar_pmid.loc[exotic_clinvar_pmid['title'].isna() == True]

In [ ]:
exotic_clinvar_exploded_pmids.loc[exotic_clinvar_exploded_pmids['citation_id'] == "10835642"]

In [ ]:
exotic_clinvar_pmid.loc[exotic_clinvar_pmid['symbol'] == 'NF1', 'citation_id']

In [ ]:
exotic_clinvar_pmid.loc[exotic_clinvar_pmid['symbol'] == 'NF1']

In [ ]:
sorted(list(exotic_clinvar_exploded_pmids.loc[exotic_clinvar_exploded_pmids['symbol'] == 'ABCC6', 'citation_id'].unique()))

In [ ]:
exotic_clinvar_pmid.citation_id.nunique()

In [ ]:
exotic_pmids_count = exotic_clinvar_pmid[['symbol', 'citation_id']].drop_duplicates().groupby('symbol').count().sort_values(by='citation_id', ascending=False)
exotic_pmids_count.to_excel('/gstock/EXOTIC/data/PMIDS/PMIDS_count_per_gene.xlsx')
exotic_pmids_count

In [ ]:
exotic_pmids_count = exotic_clinvar_pmid[['symbol', 'citation_id']].drop_duplicates()
exotic_pmids_count

In [ ]:
exotic_pmids_count = exotic_clinvar_pmid[['symbol', 'citation_id']].drop_duplicates()
d = dict(collections.Counter(exotic_pmids_count.citation_id.values.tolist()))
d = {k: d[k] for k in sorted(d, key=d.get, reverse=True)}
exotic_pmids_count = pd.DataFrame.from_dict(d, orient='index')
exotic_pmids_count.columns = ['count']
# exotic_pmids_count.to_excel('/gstock/EXOTIC/data/PMIDS/PMIDS_count_across_genes.xlsx')
# citation_to_keep = exotic_pmids_count.loc[exotic_pmids_count['count'] < 10].index.tolist()
exotic_pmids_count.head(15)




In [ ]:
nf1_publis = ["10076878", "10090487", "10336779", "10494088", "10534774", "10543400", "10607834", "10677298", "10678181", "10712197", "1071297", "10721668", "10726756", "10862084", "10980545", "11137998", "11292340", "11459867", "11476066", "11735023", "11857752", "12112660", "12483293", "12522551", "125305868", "12552569", "12707950", "12746402", "12787671", "12807981", "12822827", "12872266", "14517963", "14569132", "14722917", "15060124", "15146469", "15221447", "15627836", "1568246", "1568247", "15863657", "15948193", "16005615", "16138229", "16199547", "16380919", "16479075", "16513807", "16542390", "16544997", "16773574", "16786042", "16786508", "16835897", "16870183", "16941471", "16944272", "16961930", "17103458", "17105749", "17114577", "17160901", "17209131", "17311297", "17353900", "17369502", "17426081", "17551851", "1757093", "1770531", "17712740", "17726231", "1783401", "17914445", "17960768", "18041031", "18172006", "18183640", "18484666", "18503770", "18546366", "190611", "19061981", "19117870", "19120036", "19142971", "19221814", "19292874", "19539839", "19665063", "19738042", "19845691", "19863548", "19920235", "20142468", "20602485", "20605257", "21031597", "21089071", "2114220", "21271658", "21354044", "21362601", "21532985", "21567923", "21618341", "22034633", "22041710", "22108604", "22155606", "22190595", "22207399", "22222937", "22429592", "22604720", "22664660", "22807134", "22925204", "22965773", "23010473", "23047742", "23165953", "23222849", "23244495", "23354915", "23404336", "23460398", "23583981", "23624750", "23656349", "23668869", "23758643", "23812910", "23906300", "23913538", "23954459", "24219125", "24232412", "24357598", "24413922", "24676424", "24694336", "24711935", "24789688", "24803665", "24922668", "24932921", "24951259", "25074460", "25156439", "25211147", "25240281", "25293717", "25324428", "25324867", "25325900", "25356970", "25370043", "25403449", "25480383", "25541118", "25624686", "25810463", "25877329", "25951773", "25966637", "26056819", "26076063", "26178382", "26331193", "26345759", "26458495", "26478990", "26635368", "26659639", "26706011", "26740943", "26758488", "26840085", "26908603", "26962827", "26969325", "26973730", "27069254", "27074763", "27170677", "27171602", "27322474", "27482814", "27716896", "27838393", "27862945", "27980226", "27999334", "28068329", "28213670", "28422438", "28529006", "28706617", "28891274", "28924536", "28955729", "28961165", "29100083", "29290338", "29415745", "29449315", "29483232", "2948975", "29522274", "29566708", "29618358", "29620724", "29673180", "29685074", "29872168", "29914388", "29952103", "29957862", "30014477", "30087692", "30104415", "30190611", "30290804", "30291346", "30308447", "30530636", "31347283", "31370276", "31371350", "31533797", "31595648", "31717729", "31730495", "32860008", "6025371", "7586657", "7607663", "7649559", "7655472", "7874161", "7903661", "7904209", "7981679", "8069310", "8081387", "8242079", "8264648", "8385067", "8437860", "8544190", "8628317", "8807336", "8834249", "8837715", "8845843", "9003501", "9042399", "9101300", "9109662", "9150739", "9180088", "9195229", "9219684", "9298829", "9302992", "9375928", "9385374", "9452037", "9463322", "9475595", "9545275", "9668168", "9687500", "9783703", ]
for pmid in nf1_publis:
    print(pmid)
    url = "https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/?tool=my_tool&email=my_email@example.com&ids={}&format=json".format(pmid)
    r = requests.get(url)
    j = r.json()
    pubmed_id = j["records"][0]["pmid"]
    pmc_id = ""
    if "pmcid" in j["records"][0]:
        pmc_id= j["records"][0]["pmcid"]
    pubtator_cat = "pmc" if pmc_id else "pm"
    check = False
    if pubtator_cat == "pmc":
        try:
            print(pmc_id)
            pubtator_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?{}ids={}".format("pmc", pmc_id)
            print(pubtator_url)
            r = requests.get(pubtator_url)
            j = r.json()
            check_pheno = False
            crypto_mesh = "D003456"
            for passage in j['passages']:
                mesh_terms = [annot['infons']['identifier'] for annot in passage['annotations']]
                check_pheno = True if 'MESH:{}'.format(crypto_mesh) in mesh_terms else False
                if check_pheno:
                    print(passage)
            
            check = True if j else False
        except:
            print('PMC {} BUT NO TEXT ONLINE AVAILABLE FOR PMID : {}'.format(pmc_id, pubmed_id))

            pass
    if check is False:
        try: 
            pubtator_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?{}ids={}".format("pm", pubmed_id)
            r = requests.get(pubtator_url)
            j = r.json()
            print(pubtator_url)
            check_pheno = False
            crypto_mesh = "D003456"
            for passage in j['passages']:
                mesh_terms = [annot['infons']['identifier'] for annot in passage['annotations']]
                check_pheno = True if 'MESH:{}'.format(crypto_mesh) in mesh_terms else False
                if check_pheno:
                    print(passage)
        except:
            print('ERROR WITH {}'.format(pubmed_id))


In [ ]:
pubtator_cat = 'pmc'
pubmed_id = 'PMC1288164'
pubtator_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?{}ids={}".format(pubtator_cat, pubmed_id)
print(pubtator_url)
r = requests.get(pubtator_url)
j = r.json()
j


## OMIM entries with multiple phenotypes & at least one rare

## Processed phenotypes through API

In [ ]:
from Phenotypes import omim_associations_exotic
omim = omim_associations_exotic.load_omim(files["EXOTIC"]["omim_detailed"], files["BIOMART"]["biomart_omim"])
# omim.head()
omim

In [ ]:
l = list()
import math

for gene in tqdm(omim.Name.unique()):

    gene_omim = omim.loc[omim['Name'] == gene]
    if gene_omim.Pheno_OMIM.nunique() > 1:
#         print(gene, gene_omim.shape[0])
        gene_omim_bp = gene_omim[['Name', 'Pheno_OMIM', 'OMIM_BP']].groupby(['Name', 'Pheno_OMIM'])['OMIM_BP'].apply(set).reset_index()
        nb_pheno = gene_omim_bp.Pheno_OMIM.nunique()
        counter_pheno = collections.Counter([sub_e for e in gene_omim_bp['OMIM_BP'].values.tolist() for sub_e in e])
        cutoff = 1 if nb_pheno < 4 else math.floor(nb_pheno/2)
        counter_pheno_rare = [k for k,v in counter_pheno.items() if v <= cutoff]
#         print(nb_pheno, cutoff)
#         print(counter_pheno_rare)
        l.append(gene_omim.loc[gene_omim['OMIM_BP'].isin(counter_pheno_rare)])
        
#         for pheno_rare in counter_pheno_rare:
#             print(gene_omim.loc[gene_omim['OMIM_BP'] == pheno_rare, ['Pheno_OMIM', 'Pheno_prefered_title', 'OMIM_BP', 'OMIM_BP_phenotypes']].values.tolist())
omim_multi_pheno_filterred = pd.concat(l)
omim_multi_pheno_filterred

In [ ]:
omim_multi_pheno_filterred[['Name', 'Pheno_OMIM', 'OMIM_BP', 'OMIM_BP_phenotypes']]

In [ ]:
omim_multi_pheno_filterred[['Name', 'Pheno_OMIM']].drop_duplicates().groupby(['Name'])['Pheno_OMIM'].apply(set).reset_index()

In [ ]:
omim_multi_pheno_filterred.OMIM.nunique()

## EXOTIC imports & Filter with multi pheno OMIM genes

In [ ]:
exotic = pd.read_parquet(files["EXOTIC"]["exotic_modified_zscore_mean_prop_corrected"])
# exotic[['Exon_start', 'Exon_stop']] = exotic['Exon'].str.split('-', expand=True)
# exotic_up = omim_associations_exotic.load_exotic(files["EXOTIC"]["exotic_modified_zscore"], 'up', cutoff=0.5)
# exotic_down = omim_associations_exotic.load_exotic(files["EXOTIC"]["exotic_modified_zscore"], 'down', cutoff=0.5)
exotic = exotic.loc[exotic['symbol'].isin(omim_multi_pheno_filterred.Name.unique().tolist())]

# COMPUTE BINS
r = np.arange(0.5,1.05,0.1)
bins = r
labels = bins.copy()
labels_ratio = [str(round(labels[j], 3)) + " - " + str(round(labels[j + 1], 3)) for j in range(len(labels) - 1)]
for min_max in ['up', 'down']:
    for j, b in enumerate(labels_ratio):
        if j > 0:
            b_start = float(b.split(' - ')[0])
            b_end = float(b.split(' - ')[1])
            print(b_start)
#             print(len(exotic_clinvar_pmid[list(exotic_clinvar_pmid.columns)[9:9+53]].columns))
#             print(exotic_clinvar_pmid[list(exotic_clinvar_pmid.columns)[9:9+53]].columns)
            if min_max == 'up':
                exotic['EXOTIC_{}_{}'.format(b, min_max)] = exotic[list(exotic.columns)[9:9+53]].apply(lambda r: [exotic.columns[9:9+53][i] for i, e in enumerate(r) if float(e) >= b_start and float(e) < b_end], axis=1)
            elif min_max == 'down':
                exotic['EXOTIC_{}_{}'.format(b, min_max)] = exotic[list(exotic.columns)[9:9+53]].apply(lambda r: [exotic.columns[9:9+53][i] for i, e in enumerate(r) if (1-float(e)) >= b_start and (1-float(e)) < b_end], axis=1)
exotic


In [ ]:
print(exotic.symbol.nunique())

# Clinvar

In [ ]:
clinvar = pd.read_parquet(files['EXOTIC']['clinvar_file_path'])
clinvar = clinvar.loc[(clinvar['Status'] == 'Pathogenic') & (clinvar['Real_Status'].str.contains('onflict') == False)]
clinvar = clinvar.loc[clinvar['OMIM_VARIANT_ID'] != ""]
clinvar

In [ ]:
exotic

In [ ]:
def mp_variants(gene, exotic, clinvar, l):
    exotic_tmp_gene = exotic.loc[exotic['symbol'] == gene]
    clinvar_tmp_gene = clinvar.loc[clinvar['GENE'] == gene]
    for exon in exotic_tmp_gene.Exon.unique().tolist():
        match_variants = clinvar_tmp_gene.POS.between(-10 + int(exon.split('-')[0]), int(exon.split('-')[1]) + 10)
        match_variants = clinvar_tmp_gene.loc[match_variants.loc[match_variants == True].index.tolist(), 'OMIM_VARIANT_ID'].values.tolist()
        if match_variants:
            l.append({exon : match_variants})
        
# for gene in tqdm():
m = multiprocessing.Manager()
l = m.list()
genes = exotic.symbol.unique().tolist()
parmap.starmap(mp_variants, list(zip(genes)), exotic, clinvar, l, pm_pbar=True)
map_d = {k: v for d in l for k, v in d.items()}
exotic['OMIM_variants'] = exotic['Exon'].map(map_d)
exotic.head()

# print(l)

In [ ]:
list(exotic_omim_variants_exploded.Pheno_OMIM.values.tolist()[0])[0]

In [ ]:
dicts_omim_gtex = pd.DataFrame.from_dict(dicts['mapping_omim_gtex_neurologic'], orient='index')
dicts_omim_gtex['Pheno'] = dicts_omim_gtex.apply(lambda r: [e for e in r if e], axis=1)
dicts_omim_gtex = dicts_omim_gtex['Pheno']
dicts_omim_gtex = dicts_omim_gtex.reset_index()
dicts_omim_gtex.columns = ['value', 'Pheno']
dicts_omim_gtex = dicts_omim_gtex.explode('Pheno')


In [ ]:
exotic_omim_variants = exotic.loc[exotic['OMIM_variants'].isna() == False]
exotic_omim_variants_exploded = exotic_omim_variants.explode('OMIM_variants')
exotic_omim_variants_exploded['OMIM'] = exotic_omim_variants_exploded['OMIM_variants'].apply(lambda r: r.split('.')[0])
exotic_omim_variants_exploded['OMIM_variants'] = exotic_omim_variants_exploded['OMIM_variants'].astype(str)
exotic_omim_variants_exploded['OMIM_variant_nb'] = exotic_omim_variants_exploded['OMIM_variants'].apply(lambda r : int(r.split('.')[1]))
exotic_omim_variants_exploded = pd.merge(exotic_omim_variants_exploded, omim_multi_pheno_filterred[['Name', 'Pheno_OMIM', 'OMIM_BP', 'OMIM_BP_phenotypes']], left_on='symbol', right_on='Name')
exotic_omim_variants_exploded = exotic_omim_variants_exploded.melt(id_vars=['symbol', 'ensg', 'HGNC', 'Exon', 'Ratio_num', 'mRNA_nb', 'mRNA_nb_total', 'MAP', 'OMIM_variants', 'OMIM', 'OMIM_variant_nb', 'Name', 'Pheno_OMIM', 'OMIM_BP', 'OMIM_BP_phenotypes'],
                                  value_vars=['EXOTIC_0.6 - 0.7_up', 'EXOTIC_0.7 - 0.8_up', 'EXOTIC_0.8 - 0.9_up', 'EXOTIC_0.9 - 1.0_up', 'EXOTIC_0.6 - 0.7_down', 'EXOTIC_0.7 - 0.8_down', 'EXOTIC_0.8 - 0.9_down', 'EXOTIC_0.9 - 1.0_down']).explode('value').dropna(subset=['value'])
print(exotic_omim_variants_exploded.shape)
exotic_omim_variants_exploded.head()
exotic_omim_variants_exploded = pd.merge(exotic_omim_variants_exploded, dicts_omim_gtex, on='value')
exotic_omim_variants_exploded = exotic_omim_variants_exploded.loc[exotic_omim_variants_exploded['OMIM_BP'] == exotic_omim_variants_exploded['Pheno']]
exotic_omim_variants_exploded = pd.merge(exotic_omim_variants_exploded.rename({'OMIM_variants' : 'OMIM_VARIANT_ID'}, axis=1), clinvar[['VAR_ID', 'MC', 'Real_Status', 'RS_STARS', 'CLNREVSTAT', 'alleleid', 'OMIM_VARIANT_ID']], on='OMIM_VARIANT_ID')
exotic_omim_variants_exploded

In [ ]:
import _pickle
import re
# for gene in exotic_omim_variants_exploded['OMIM'].unique()[:1]:
def retrieve_omim_variants_info(r):
#     print(r)
#     print(r['Pheno_OMIM'])
    omim_gene_json = _pickle.load(open('/gstock/biolo_datasets/variation/benchmark/Databases/OMIM/JSON_API/{}.pkl'.format(str(r['OMIM'])), 'rb'))
    variant_json = omim_gene_json['omim']['entryList'][0]['entry']['allelicVariantList'][r['OMIM_variant_nb'] - 1]['allelicVariant']
#     print(variant_json)
    if variant_json['text'].startswith('See {') is False:

        pheno_id = [publi for publi in re.findall('\{.*?\}', variant_json['text']) if ':' not in publi and  '.' not in publi]
        if pheno_id:
            pheno_id = pheno_id[0]
            pheno_id = int(pheno_id.replace('{', '').replace('}', ''))
            publi_ids = [publi for publi in re.findall('\{.*?\}', variant_json['text']) if ':' in publi]
            publi_ids = [sub_publi for publi in publi_ids for sub_publi in publi.split(':')[0].replace('{', '').split(',')]

            
            if pheno_id == r['Pheno_OMIM']:
                pmids = []
                try:
                    pmids = [omim_gene_json['omim']['entryList'][0]['entry']['referenceList'][int(publi.split(':')[0].replace('{', '')) - 1]['reference']['pubmedID'] for publi in publi_ids]
                except:
#                     pprint([omim_gene_json['omim']['entryList'][0]['entry']['referenceList'][int(publi.split(':')[0].replace('{', '')) - 1]['reference'] for publi in publi_ids])
                    pass
                return pmids
# exotic_omim_variants_exploded.loc[exotic_omim_variants_exploded['symbol'] == 'TP63'].apply(lambda r: retrieve_omim_variants_info(r), axis=1)   
exotic_omim_variants_exploded['PMIDS_OMIM'] = exotic_omim_variants_exploded.apply(lambda r: retrieve_omim_variants_info(r), axis=1)    
exotic_omim_variants_exploded

    


In [ ]:
exotic_omim_variants_exploded.symbol.nunique()

In [ ]:
sorted(exotic_omim_variants_exploded.symbol.unique().tolist())

In [ ]:
pd.options.display.max_rows = 515
exotic_omim_variants_exploded = exotic_omim_variants_exploded.loc[exotic_omim_variants_exploded['PMIDS_OMIM'].isna() == False]
exotic_omim_variants_exploded.loc[exotic_omim_variants_exploded['symbol'] != 'PTEN'].sort_values(by='symbol', ascending=True).to_excel('/gstock/EXOTIC/data/PHENOTYPES/exotic_omim_variants_filtered.xlsx', index=False)

In [ ]:
exotic.loc[exotic['symbol'] == 'LRP5']

In [ ]:
exotic_omim_variants_exploded.symbol.nunique()

In [ ]:
pd.options.display.max_rows = 400
exotic_omim_variants_exploded[[
    'symbol', 'Exon', 'EXOTIC_0.6 - 0.7_up', 'EXOTIC_0.7 - 0.8_up', 'EXOTIC_0.8 - 0.9_up',
       'EXOTIC_0.9 - 1.0_up', 'EXOTIC_0.6 - 0.7_down', 'EXOTIC_0.7 - 0.8_down',
       'EXOTIC_0.8 - 0.9_down', 'EXOTIC_0.9 - 1.0_down', 'PMIDS_OMIM', 'Pheno_OMIM', 'OMIM_variants'
]]

# Complete table to build inverted pyramid

## EXOTIC imports

In [ ]:
exotic = pd.read_parquet(files["EXOTIC"]["exotic_modified_zscore"])
exotic[['Exon_start', 'Exon_stop']] = exotic['Exon'].str.split('-', expand=True)
# exotic_up = omim_associations_exotic.load_exotic(files["EXOTIC"]["exotic_modified_zscore"], 'up', cutoff=0.5)
# exotic_down = omim_associations_exotic.load_exotic(files["EXOTIC"]["exotic_modified_zscore"], 'down', cutoff=0.5)

# Clinvar

In [ ]:
clinvar = pd.read_parquet(files['EXOTIC']['clinvar_file_path'])
clinvar = clinvar.loc[(clinvar['Status'] == 'Pathogenic') & (clinvar['Real_Status'].str.contains('onflict') == False)]
# clinvar = clinvar.loc[clinvar['OMIM_VARIANT_ID'] != ""]
# clinvar['ALT_Lite'] = clinvar['ALT'].apply(lambda r: eval(r)[0]) 
# clinvar['VAR_ID_Lite'] = clinvar['CHROM'].astype(str) + '_' + clinvar['POS'].astype(str) + '_' + clinvar['REF'].astype(str) + '_' + clinvar['ALT_Lite'].astype(str)

clinvar

In [ ]:
def mp_variants(gene, exotic, clinvar, l):
    exotic_tmp_gene = exotic.loc[exotic['symbol'] == gene]
    clinvar_tmp_gene = clinvar.loc[clinvar['GENE'] == gene]
    for exon in exotic_tmp_gene.Exon.unique().tolist():
        match_variants = clinvar_tmp_gene.POS.between(int(exon.split('-')[0]), int(exon.split('-')[1]))
        match_variants = clinvar_tmp_gene.loc[match_variants.loc[match_variants == True].index.tolist(), 'alleleid'].values.tolist()
        if match_variants:
            l.append({exon : match_variants})
        
# for gene in tqdm():
m = multiprocessing.Manager()
l = m.list()
genes = exotic.symbol.unique().tolist()
parmap.starmap(mp_variants, list(zip(genes)), exotic, clinvar, l, pm_pbar=True)
map_d = {k: v for d in l for k, v in d.items()}
exotic['AlleleIDs_clinvar'] = exotic['Exon'].map(map_d)
# exotic_up['AlleleIDs_clinvar'] = exotic_up['Exon'].map(map_d)
# exotic_down['AlleleIDs_clinvar'] = exotic_down['Exon'].map(map_d)


# print(l)

In [ ]:
exotic.loc[exotic['AlleleIDs_clinvar'].isna() == False, 'symbol'].nunique()

In [ ]:
exotic_clinvar = pd.merge(exotic.explode('AlleleIDs_clinvar').rename({'AlleleIDs_clinvar' : 'alleleid'}, axis=1), clinvar, on='alleleid')
exotic_clinvar['OMIM'] = exotic_clinvar['OMIM_VARIANT_ID'].apply(lambda r: str(r).split('.')[0])
exotic_clinvar.loc[exotic_clinvar['OMIM_VARIANT_ID'] != '', 'OMIM_variant_nb'] = exotic_clinvar.loc[exotic_clinvar['OMIM_VARIANT_ID'] != '']['OMIM_VARIANT_ID'].apply(lambda r: int(str(r).split('.')[1]))
exotic_clinvar.to_csv('/gstock/EXOTIC/data/VARIATIONS/exotic_clinvar.csv.gz', compression='gzip', sep='\t', index=False)
exotic_clinvar.to_excel('/gstock/EXOTIC/data/VARIATIONS/exotic_clinvar.xlsx', index=False)
exotic_clinvar

In [ ]:
exotic_clinvar

In [ ]:
exotic_clinvar = pd.read_csv('/gstock/EXOTIC/data/VARIATIONS/exotic_clinvar.csv.gz', compression='gzip', sep='\t')
exotic_clinvar['OMIM'] = exotic_clinvar['OMIM'].astype(int)
# exotic_clinvar.loc[exotic_clinvar['OMIM_VARIANT_ID'] != '', 'OMIM_variant_nb'] = exotic_clinvar.loc[exotic_clinvar['OMIM_VARIANT_ID'] != '']['OMIM_VARIANT_ID'].apply(lambda r: int(str(r).split('.')[1]))
exotic_clinvar


In [ ]:
exotic_clinvar.columns

In [ ]:
import _pickle
import re
# for gene in exotic_omim_variants_exploded['OMIM'].unique()[:1]:
l = list()

def forbid(r):
    forbidden = [':', '.', 'rs', 'ss', 'dbSNP']
    check = True if set([True if e not in r else False for e in forbidden]) == {True} else False
    return check 

def retrieve_omim_variants_info(r):
#     print(r)
#     print(r['Pheno_OMIM'])
#     try:
    if r['OMIM']:
        omim_gene_json = _pickle.load(open('/gstock/biolo_datasets/variation/benchmark/Databases/OMIM/JSON_API/{}.pkl'.format(str(r['OMIM'])), 'rb'))
        if 'allelicVariantList' in  omim_gene_json['omim']['entryList'][0]['entry']:
            variant_json = omim_gene_json['omim']['entryList'][0]['entry']['allelicVariantList'][r['OMIM_variant_nb'] - 1]['allelicVariant']
#             print(variant_json)
        #     print(variant_json)
            if variant_json['text'].startswith('See {') is False:
                
                pheno_ids = [publi for publi in re.findall('\{.*?\}', variant_json['text']) if forbid(publi) is True]
    #             pheno_id = pheno_id[0]
                pheno_ids = [int(pheno.replace('{', '').replace('}', '')) for pheno in pheno_ids]
                publi_ids = [publi for publi in re.findall('\{.*?\}', variant_json['text']) if ':' in publi]
                publi_ids = [sub_publi for publi in publi_ids for sub_publi in publi.split(':')[0].replace('{', '').split(',')]
    #             print(r, pheno_ids)


    #             if pheno_id == r['Pheno_OMIM']:
                pmids = []
                try:
                    pmids = [omim_gene_json['omim']['entryList'][0]['entry']['referenceList'][int(publi.split(':')[0].replace('{', '')) - 1]['reference']['pubmedID'] for publi in publi_ids]
                    
                except:
    #                     pprint([omim_gene_json['omim']['entryList'][0]['entry']['referenceList'][int(publi.split(':')[0].replace('{', '')) - 1]['reference'] for publi in publi_ids])
                    pass
                r['PMIDS_OMIM'] = pmids
                r['PHENOS_OMIM'] = pheno_ids
    return r
                
#                     l.append(r['OMIM'])
#     except:
#         l.append(r['OMIM'])
            
                
# exotic_omim_variants_exploded.loc[exotic_omim_variants_exploded['symbol'] == 'TP63'].apply(lambda r: retrieve_omim_variants_info(r), axis=1) 
exotic_clinvar_tmp  = exotic_clinvar.dropna(subset=['OMIM', 'OMIM_variant_nb'])
exotic_clinvar_tmp['OMIM_variant_nb'] = exotic_clinvar_tmp['OMIM_variant_nb'].astype(int)
exotic_clinvar_tmp['OMIM'] = exotic_clinvar_tmp['OMIM'].astype(int)

exotic_clinvar_tmp = exotic_clinvar_tmp.apply(retrieve_omim_variants_info, axis=1)
exotic_clinvar_tmp = exotic_clinvar_tmp[list(exotic_clinvar.columns) + ['PMIDS_OMIM', 'PHENOS_OMIM']]
exotic_clinvar_tmp
# print(len(l))
# print(l)

    


In [ ]:
exotic_clinvar_complete = pd.concat([exotic_clinvar_tmp, exotic_clinvar.loc[exotic_clinvar['OMIM'].isna() == True]], axis=0).sort_values(by='MAP')
# test.loc[test['symbol'].isna() == True]
print(exotic_clinvar_complete.symbol.nunique())
exotic_clinvar_complete

## OMIM entries with multiple phenotypes & at least one rare

In [ ]:
def load_omim(omim_path, biomart_omim_path):
    omim = pd.read_csv(
        omim_path,
        compression="gzip",
        sep="\t",
    )
#     return omim

    print("Total : ", omim.OMIM.nunique())
    omim = omim.dropna(subset=list(omim.columns[6:-2]), how="all")
    print("Dropna on all cols : ", omim.OMIM.nunique())

    biomart_omim = pd.read_csv(biomart_omim_path, sep="\t", compression="gzip").dropna(subset=["MIM gene accession"])
    biomart_omim["MIM gene accession"] = biomart_omim["MIM gene accession"].astype(int)
    biomart_omim = biomart_omim.rename({"MIM gene accession": "OMIM", "Gene stable ID": "ensg", "Gene name": "Name"}, axis=1)

    ## ADD Gene TO OMIM
    omim = pd.merge(biomart_omim[["ensg", "Name", "OMIM"]], omim, on="OMIM")
    print("Merge BIOMART : ", omim.OMIM.nunique())


    # MELT
    omim = omim.melt(id_vars=list(omim.columns)[:7], value_vars=list(omim.columns)[7:], var_name="OMIM_BP", value_name="OMIM_BP_phenotypes").dropna()

    return omim
omim = load_omim(files["EXOTIC"]["omim_detailed"], files["BIOMART"]["biomart_omim"])
omim

In [ ]:
l = list()
for gene in tqdm(omim.Name.unique()):

    gene_omim = omim.loc[omim['Name'] == gene]
    if gene_omim.Pheno_OMIM.nunique() > 1:
#         print(gene, gene_omim.shape[0])
        gene_omim_bp = gene_omim[['Name', 'Pheno_OMIM', 'OMIM_BP']].groupby(['Name', 'Pheno_OMIM'])['OMIM_BP'].apply(set).reset_index()
        counter_pheno = collections.Counter([sub_e for e in gene_omim_bp['OMIM_BP'].values.tolist() for sub_e in e])
        counter_pheno_rare = [k for k,v in counter_pheno.items() if v == 1]
        l.append(gene_omim.loc[gene_omim['OMIM_BP'].isin(counter_pheno_rare)])
        
#         for pheno_rare in counter_pheno_rare:
#             print(gene_omim.loc[gene_omim['OMIM_BP'] == pheno_rare, ['Pheno_OMIM', 'Pheno_prefered_title', 'OMIM_BP', 'OMIM_BP_phenotypes']].values.tolist())
omim_multi_pheno_filterred = pd.concat(l)

omim_multi_pheno_filterred

In [ ]:
list_bp_phenos_diff = omim_multi_pheno_filterred.groupby('Pheno_OMIM')['OMIM_BP'].apply(set).reset_index()
list_bp_phenos_diff.columns = ['PHENOS_OMIM', 'BP_DIFF']



In [ ]:
omim_pheno_diff = omim_multi_pheno_filterred.Pheno_OMIM.unique().tolist()

In [ ]:
# aexotic_clinvar_complete.loc[exotic_clinvar_complete['PHENOS_OMIM'].isna() == False]
exotic_clinvar_complete = exotic_clinvar_complete.explode('PHENOS_OMIM')
exotic_clinvar_complete['PHENOS_DIFF'] = exotic_clinvar_complete['PHENOS_OMIM'].apply(lambda r: r in omim_pheno_diff)
exotic_clinvar_complete = pd.merge(exotic_clinvar_complete, list_bp_phenos_diff, on='PHENOS_OMIM', how='left')
exotic_clinvar_complete

In [ ]:
dicts = json.load(open("/home/weber/PycharmProjects/EXOTIC/src/EXOTIC_config.json"))
mapping_omim_gtex = dicts["mapping_omim_gtex_neurologic"]

In [ ]:
tissues = ['Adipose - Subcutaneous', 'Adipose - Visceral (Omentum)', 'Adrenal Gland', 'Artery - Aorta', 'Artery - Coronary', 'Artery - Tibial', 'Bladder', 'Brain - Amygdala', 'Brain - Anterior cingulate cortex (BA24)', 'Brain - Caudate (basal ganglia)', 'Brain - Cerebellar Hemisphere', 'Brain - Cerebellum', 'Brain - Cortex', 'Brain - Frontal Cortex (BA9)', 'Brain - Hippocampus', 'Brain - Hypothalamus', 'Brain - Nucleus accumbens (basal ganglia)', 'Brain - Putamen (basal ganglia)', 'Brain - Spinal cord (cervical c-1)', 'Brain - Substantia nigra', 'Breast - Mammary Tissue', 'Cells - Cultured fibroblasts', 'Cells - EBV-transformed lymphocytes', 'Cervix - Ectocervix', 'Cervix - Endocervix', 'Colon - Sigmoid', 'Colon - Transverse', 'Esophagus - Gastroesophageal Junction', 'Esophagus - Mucosa', 'Esophagus - Muscularis', 'Fallopian Tube', 'Heart - Atrial Appendage', 'Heart - Left Ventricle', 'Kidney - Cortex', 'Liver', 'Lung', 'Minor Salivary Gland', 'Muscle - Skeletal', 'Nerve - Tibial', 'Ovary', 'Pancreas', 'Pituitary', 'Prostate', 'Skin - Not Sun Exposed (Suprapubic)', 'Skin - Sun Exposed (Lower leg)', 'Small Intestine - Terminal Ileum', 'Spleen', 'Stomach', 'Testis', 'Thyroid', 'Uterus', 'Vagina', 'Whole Blood']
exotic_clinvar_complete['Up_above_cutoff'] = exotic_clinvar_complete[tissues].apply(lambda r: [tissues[j] for j, e in enumerate(r) if e > 0.6], axis=1)
exotic_clinvar_complete['Up_tissues_EXOTIC_values'] = exotic_clinvar_complete.apply(lambda r: [round(r[e], 3) for e in r['Up_above_cutoff']], axis=1)
exotic_clinvar_complete['BP_Up'] = exotic_clinvar_complete['Up_above_cutoff'].apply(lambda r : [bp for e in r if e in mapping_omim_gtex for bp in mapping_omim_gtex[e] ])
exotic_clinvar_complete['Down_above_cutoff'] = exotic_clinvar_complete[tissues].apply(lambda r: [tissues[j] for j, e in enumerate(r) if e < 0.4], axis=1)
exotic_clinvar_complete['Down_tissues_EXOTIC_values'] = exotic_clinvar_complete.apply(lambda r: [round(float(1 - r[e]), 3) for e in r['Down_above_cutoff']], axis=1)

exotic_clinvar_complete['BP_Down'] = exotic_clinvar_complete['Down_above_cutoff'].apply(lambda r : [bp for e in r if e in mapping_omim_gtex for bp in mapping_omim_gtex[e] ])
exotic_clinvar_complete.loc[(exotic_clinvar_complete['BP_Up'].isna() == False) & (exotic_clinvar_complete['BP_DIFF'].isna() == False), 'Check_BP_up'] = exotic_clinvar_complete.loc[(exotic_clinvar_complete['BP_Up'].isna() == False) & (exotic_clinvar_complete['BP_DIFF'].isna() == False)].apply(lambda r: [e  for e in r['BP_Up'] if e  and e in  r['BP_DIFF']], axis=1)
exotic_clinvar_complete.loc[(exotic_clinvar_complete['BP_Down'].isna() == False) & (exotic_clinvar_complete['BP_DIFF'].isna() == False), 'Check_BP_down'] = exotic_clinvar_complete.loc[(exotic_clinvar_complete['BP_Down'].isna() == False) & (exotic_clinvar_complete['BP_DIFF'].isna() == False)].apply(lambda r: [e  for e in r['BP_Down'] if e  and e in  r['BP_DIFF']], axis=1)

exotic_clinvar_complete


In [ ]:
exotic_clinvar_complete.to_csv('/gstock/EXOTIC/data/VARIATIONS/EXOTIC_clinvar_OMIM.csv.gz', compression='gzip', sep='\t', index=False)
exotic_clinvar_complete.to_excel('/gstock/EXOTIC/data/VARIATIONS/EXOTIC_clinvar_OMIM.xlsx', index=False)
exotic_clinvar_complete

In [ ]:
exotic_clinvar_complete.columns

## Stats

In [ ]:
list_cols = ['OMIM_VARIANT_ID', 'PHENOS_OMIM', 'PHENOS_DIFF', 'PMIDS_OMIM']
for j, col in enumerate(list_cols):
    included_cols = list_cols[:j+1]
    tmp_exotic = exotic_clinvar_complete[['symbol'] + included_cols]
    if 'PHENOS_DIFF' in included_cols:
        tmp_exotic = tmp_exotic.loc[tmp_exotic['PHENOS_DIFF'] == True]
    print(j, col, included_cols, tmp_exotic.dropna()['symbol'].nunique())
    
    
    

In [ ]:
exotic_clinvar_complete.symbol.nunique()

In [ ]:
list_cols = ['OMIM_VARIANT_ID', 'PHENOS_OMIM', 'PHENOS_DIFF', 'PMIDS_OMIM', 'Up_above_cutoff', 'BP_Up', 'Up_tissues_EXOTIC_values', 'Check_BP_up', ]
for j, col in enumerate(list_cols):
    included_cols = list_cols[:j+1]
    tmp_exotic = exotic_clinvar_complete[['symbol'] + included_cols].dropna()
    if 'PHENOS_DIFF' in included_cols:
        tmp_exotic = tmp_exotic.loc[tmp_exotic['PHENOS_DIFF'] == True]
    if 'up' in col.lower():
        tmp_exotic = tmp_exotic.loc[tmp_exotic[col].str.len() > 0]

    print(j, col, included_cols, tmp_exotic['symbol'].nunique())
    
tmp_exotic.loc[4552].to_dict()

In [ ]:
list_cols = ['OMIM_VARIANT_ID', 'PHENOS_OMIM', 'PHENOS_DIFF', 'PMIDS_OMIM', 'Down_above_cutoff', 'BP_Down', 'Down_tissues_EXOTIC_values', 'Check_BP_down', ]
for j, col in enumerate(list_cols):
    included_cols = list_cols[:j+1]
    tmp_exotic = exotic_clinvar_complete[['symbol'] + included_cols].dropna()
    if 'PHENOS_DIFF' in included_cols:
        tmp_exotic = tmp_exotic.loc[tmp_exotic['PHENOS_DIFF'] == True]
    if 'down' in col.lower():
        tmp_exotic = tmp_exotic.loc[tmp_exotic[col].str.len() > 0]

    print(j, col, included_cols, tmp_exotic['symbol'].nunique())
pd.options.display.max_rows = 70
tmp_exotic
    

In [ ]:
list_cols = ['OMIM_VARIANT_ID', 'PHENOS_OMIM', 'PHENOS_DIFF', 'PMIDS_OMIM']

for cols in [['Down_above_cutoff', 'Up_above_cutoff'], ['BP_Up', 'BP_Down'], ['Check_BP_up', 'Check_BP_down']]:
    tmp_exotic_down = exotic_clinvar_complete[['symbol'] + list_cols + [cols[0]]].dropna()
    tmp_exotic_down = tmp_exotic_down.loc[tmp_exotic_down['PHENOS_DIFF'] == True]
    tmp_exotic_down = tmp_exotic_down.loc[tmp_exotic_down[cols[0]].str.len() > 0]


    tmp_exotic_up = exotic_clinvar_complete[['symbol'] + list_cols + [cols[1]]].dropna()
    tmp_exotic_up = tmp_exotic_up.loc[tmp_exotic_up['PHENOS_DIFF'] == True]
    tmp_exotic_up = tmp_exotic_up.loc[tmp_exotic_up[cols[1]].str.len() > 0]


    print(cols, len(set(tmp_exotic_up.symbol.unique().tolist() + tmp_exotic_down.symbol.unique().tolist())))

